In [25]:
import pandas as pd
import numpy as np

df = pd.read_json('data/normalized_tweets.json')

### Make a column contains word and the hour its posted

In [26]:
df['hour'] = df.datetime.apply(lambda d: d.hour)
df['word'] = df.message.apply(lambda m: m.split(' '))

In [27]:
df.head(10)

,datetime,link,message,hour,word
0,2017-12-18 22:01:15,https://twitter.com/jokowi/status/942771750827...,anak muda entrepreneurs pengalaman pribadi sus...,22,"[anak, muda, entrepreneurs, pengalaman, pribad..."
1,2017-12-18 16:19:09,https://twitter.com/jokowi/status/942685658312...,bandung ketemu duo perempuan hebat indonesia m...,16,"[bandung, ketemu, duo, perempuan, hebat, indon..."
2,2017-12-18 11:36:41,https://twitter.com/jokowi/status/942614572673...,negara hadir melindungi buruh migran indonesia...,11,"[negara, hadir, melindungi, buruh, migran, ind..."
3,2017-12-16 22:11:24,https://twitter.com/jokowi/status/942049529460...,mengantar habibie kediamannya komplek patra ku...,22,"[mengantar, habibie, kediamannya, komplek, pat..."
4,2017-12-16 20:48:38,https://twitter.com/jokowi/status/942028699687...,negara dikelilingi titik gempa waspada siaga r...,20,"[negara, dikelilingi, titik, gempa, waspada, s..."
5,2017-12-15 17:41:33,https://twitter.com/jokowi/status/941619231539...,pembangunan jalur ganda kereta api bogorsukabu...,17,"[pembangunan, jalur, ganda, kereta, api, bogor..."
6,2017-12-14 13:22:18,https://twitter.com/jokowi/status/941191598007...,inna lillahi wa inna ilaihi rojiunkita kehilan...,13,"[inna, lillahi, wa, inna, ilaihi, rojiunkita, ..."
7,2017-12-13 20:13:11,https://twitter.com/jokowi/status/940932613253...,mengajak negara anggota organisasi kerja islam...,20,"[mengajak, negara, anggota, organisasi, kerja,..."
8,2017-12-11 19:57:21,https://twitter.com/jokowi/status/940203853617...,regulasi pisau bermata aturan izin persyaratan...,19,"[regulasi, pisau, bermata, aturan, izin, persy..."
9,2017-12-11 07:45:34,https://twitter.com/jokowi/status/940019696295...,emang enak nggak radio joko widodo pendengar r...,7,"[emang, enak, nggak, radio, joko, widodo, pend..."


### Labeling words with hour its posted and splitting them into their own records

In [28]:
df = df.set_index(['hour'])['word'].apply(pd.Series).stack()
df = df.reset_index()
df.drop('level_1', axis=1, inplace=True)
df.columns = ['hour', 'word']
df.head(10)

,hour,word
0,22,anak
1,22,muda
2,22,entrepreneurs
3,22,pengalaman
4,22,pribadi
5,22,susahnya
6,22,cari
7,22,modal
8,22,usaha
9,22,pembeli


### grouping words by hour and counting them

In [29]:
df = df.groupby(['hour', 'word']).size().reset_index(name='total')

sampling word 'bangsa'

In [30]:
df[df.word=='bangsa']

,hour,word,total
229,5,bangsa,1
265,6,bangsa,2
590,8,bangsa,1
873,9,bangsa,2
1177,10,bangsa,1
1507,11,bangsa,3
1762,12,bangsa,3
2037,13,bangsa,1
2365,14,bangsa,3
2965,16,bangsa,1


#### filling hour with zero word usage / fill na with 0
#### building json dictionary

In [31]:
word_list = list(set(df.word))
result_dict = []

hour_df = pd.DataFrame(np.arange(24))
hour_df['total'] = 0
hour_df.columns = ['hour', 'total']

for word in word_list:
    dt = df[df.word==word]
    dt = dt[['hour', 'total']]
    
    dx = pd.merge(hour_df, dt, on='hour', how='left')
    dx = dx[['hour', 'total_y']]
    dx.columns = ['hours', 'total']
    dx = dx.fillna(0)
    dx.total = dx.total.astype(int)    
    
    obj = {'word': word, 'data': dx, 'total': dx.total.sum()}
    result_dict.append(obj)


### writting json result

In [36]:
dm = pd.DataFrame(result_dict)
dm = dm.sort_values(by=['total'], ascending=False)[:100]

with open('results/total_word_in_every_hour.json', 'w', encoding='utf-8') as outfile:
    outfile.write(dm.to_json(orient='records'))